In [1]:
import torch
device = 'cpu'
import matplotlib.pyplot as plt
import numpy as np
from models.training import create_dataloader


# Juptyer magic: For export. Makes the plots size right for the screen 
%matplotlib inline
# %config InlineBackend.figure_format = 'retina'

%config InlineBackend.figure_formats = ['svg'] 

torch.backends.cudnn.deterministic = True
seed = np.random.randint(1,200)
seed = 57
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
print(seed)
g = torch.Generator()
g.manual_seed(seed)

# design choices 
chosen_problem = 'restricted_TS'
data_noise = 0.
n_different_weights = 1
if n_different_weights == 1:
    print('This choice will generate autonomous dynamics')
else:
    print('This choice generates non-autonomous dynamics, letting the weights depend on time')

possible_problem = {'moons':'moons', 'ToggleSwitch':'TS', 'repressilator':'repr', 'restricted_TS': 'restrictedTS'} 
# this choices determine the data set that we build and subsequent choices on the construction of the neural ODE 
# - in particular, it determines the dimensions 
problem = possible_problem[chosen_problem]

plotlim = [0, 5]

#Import of the model dynamics that describe the neural ODE
#The dynamics are based on the torchdiffeq package, that implements ODE solvers in the pytorch setting
from models.neural_odes import NeuralODE

#T is the end time of the neural ODE evolution, num_steps are the amount of discretization steps for the ODE solver
T, num_steps = 1, n_different_weights
bound = 0.
fp = False
cross_entropy = False
turnpike = False

# choice of model: what nonlinearity is used and if the nonlinearity is applied before (inside) or after (outside) the linear weights
# another choice is bottleneck, but I don't understand it
# non_linearity = 'tanh' # OR 'relu' 'sigmoid' 'leakyrelu' 'tanh_prime'
# architecture = 'inside' 'outside'
non_linearity = 'tanh'
architecture = 'restricted'
architectures = {'inside': -1, 'outside': 0, 'bottleneck': 1, 'restricted': 2}
# number of optimization runs in which the dataset is used for gradient decent
num_epochs = 50
if problem == 'moons' or problem == 'TS' or problem == "restrictedTS":
    hidden_dim, data_dim = 2, 2 
else:
    hidden_dim, data_dim = 3, 3 
augment_dim = 0

57
This choice will generate autonomous dynamics


In [2]:
W1 = []
for k in range(0,20):
    
    dataloader, dataloader_viz = create_dataloader(problem, batch_size = 2, noise = data_noise, 
                                               plotlim = plotlim, random_state = k, label = 'vector')
    
    # resets the seed - allows for coherent runs in the gradient descent as well
    torch.manual_seed(k)
    torch.cuda.manual_seed(k)
    anode = NeuralODE(device, data_dim, hidden_dim, output_dim=data_dim, augment_dim=augment_dim, non_linearity=non_linearity, 
                        architecture=architecture, T=T, time_steps=num_steps, fixed_projector=fp, cross_entropy=cross_entropy)
    optimizer_anode = torch.optim.Adam(anode.parameters(), lr=1e-1)

    from models.training import doublebackTrainer

    torch.manual_seed(k)
    torch.cuda.manual_seed(k)
    trainer_anode = doublebackTrainer(anode, optimizer_anode, device, cross_entropy=cross_entropy, turnpike = turnpike,
                            bound=bound, fixed_projector=fp, verbose = True, eps_comp = 0.2)
    
    trainer_anode.train(dataloader, 100)
    W1.append(anode.flow.dynamics.fc1_time[0].weight)

No change  applied to TS or repr data
Epoch 1: 6.484
Epoch 2: 5.610
Epoch 3: 5.202
Epoch 4: 5.086
Epoch 5: 4.258
Epoch 6: 3.203
Epoch 7: 2.421
Epoch 8: 2.019
Epoch 9: 1.879
Epoch 10: 1.867
Epoch 11: 1.895
Epoch 12: 1.925
Epoch 13: 1.946
Epoch 14: 1.955
Epoch 15: 1.957
Epoch 16: 1.955
Epoch 17: 1.947
Epoch 18: 1.933
Epoch 19: 1.912
Epoch 20: 1.879
Epoch 21: 1.832
Epoch 22: 1.768
Epoch 23: 1.685
Epoch 24: 1.582
Epoch 25: 1.461
Epoch 26: 1.329
Epoch 27: 1.191
Epoch 28: 1.049
Epoch 29: 0.891
Epoch 30: 0.704
Epoch 31: 0.507
Epoch 32: 0.345
Epoch 33: 0.234
Epoch 34: 0.174
Epoch 35: 0.154
Epoch 36: 0.151
Epoch 37: 0.125
Epoch 38: 0.124
Epoch 39: 0.146
Epoch 40: 0.118
Epoch 41: 0.139
Epoch 42: 0.148
Epoch 43: 0.139
Epoch 44: 0.164
Epoch 45: 0.155
Epoch 46: 0.168
Epoch 47: 0.176
Epoch 48: 0.170
Epoch 49: 0.180
Epoch 50: 0.175
Epoch 51: 0.174
Epoch 52: 0.175
Epoch 53: 0.166
Epoch 54: 0.160
Epoch 55: 0.156
Epoch 56: 0.144
Epoch 57: 0.140
Epoch 58: 0.132
Epoch 59: 0.123
Epoch 60: 0.121
Epoch 61: 0

In [3]:
print(W1)

[Parameter containing:
tensor([[-0.0621, -0.9080],
        [-1.2906,  0.4869]], requires_grad=True), Parameter containing:
tensor([[-0.0077, -0.9886],
        [-0.9806, -0.0014]], requires_grad=True), Parameter containing:
tensor([[-1.2060e-02, -9.8817e-01],
        [-9.9006e-01,  1.1185e-04]], requires_grad=True), Parameter containing:
tensor([[-0.0088, -0.9925],
        [-0.9877, -0.0072]], requires_grad=True), Parameter containing:
tensor([[-0.0819, -0.9170],
        [-1.1492,  0.1501]], requires_grad=True), Parameter containing:
tensor([[ 0.0021, -1.0095],
        [-0.9838, -0.0230]], requires_grad=True), Parameter containing:
tensor([[-0.0113, -1.0047],
        [-1.0128, -0.0017]], requires_grad=True), Parameter containing:
tensor([[-0.0048, -0.9957],
        [-0.9928, -0.0058]], requires_grad=True), Parameter containing:
tensor([[-0.1060, -0.8627],
        [-1.3749,  0.5978]], requires_grad=True), Parameter containing:
tensor([[-0.0108, -0.9932],
        [-0.9913, -0.0074]], requ